In [2]:
from jetcam.usb_camera import USBCamera
from jetcam.csi_camera import CSICamera
import torchvision.transforms as transforms
import ipywidgets
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
import os
from uuid import uuid1

In [3]:
# Check device number
!ls -ltrh /dev/video*

crw-rw----+ 1 root video 81, 0 Thg 8 14 18:33 /dev/video0


In [4]:
# for USB Camera (Logitech C270 webcam), uncomment the following line
camera = USBCamera(width=224, height=224, capture_device=0) # confirm the capture_device number

# for CSI Camera (Raspberry Pi Camera Module V2), uncomment the following line
# camera = CSICamera(width=224, height=224, capture_device=0) # confirm the capture_device number

camera.running = True
print("camera created")

camera created


In [5]:
TRANSFORMS = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

LABELS = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'empty']

In [6]:
DATA_DIR = 'dataset-more'

In [7]:
DATA_DIR = 'dataset-less'

In [9]:
!mkdir -p {DATA_DIR}
for label in LABELS:
    !mkdir -p {DATA_DIR + '/' + label}

In [18]:
# unobserve all callbacks from camera in case we are running this cell for second time
camera.unobserve_all()

# create image preview
camera_widget = ipywidgets.Image()
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# create widgets
category_widget = ipywidgets.Dropdown(options=LABELS, description='category')
count_widget = ipywidgets.IntText(description='count')
save_widget = ipywidgets.Button(description='add')

# manually update counts at initialization
count_widget.value = len(os.listdir(DATA_DIR + '/' + LABELS[0]))

# update counts when we select a new category
def update_counts(change):
    count_widget.value = len(os.listdir(DATA_DIR + '/' + change['new']))
category_widget.observe(update_counts, names='value')

# save image for category and update counts
def save(c):
    image_path = os.path.join(DATA_DIR + '/' + category_widget.value, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(camera_widget.value)
    count_widget.value = len(os.listdir(DATA_DIR + '/' + category_widget.value))

save_widget.on_click(save)

data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget]), category_widget, count_widget, save_widget
])

# display(data_collection_widget)
print("data_collection_widget created")

data_collection_widget created


In [19]:
display(data_collection_widget)